# buidling quantum CNN circuit
1. **transforme data to quantum states**

2. **transfrome Classical CNN model to Quantum CNN**

3. **add multi scale renormalization anstaz** : lhaja lowla hna c ndirou short range disentangler b CNOT nahiw biha el uneccecary entanglement bin consucutive qubits , apres napplyaiw RY gate bach nfusioniw two qubits l one larger information (qubits ytkhyrou 3la hssab architecture ta3 mera) , ouraha CNOT whdokhr ynahi el entanglement bin larger infos hadouma ,

4. **add quantum error correction** :

5. **test** :


In [18]:
!pip install pennylane

In [19]:
import pennylane as qml
from pennylane import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

n_qubits = 6
dev = qml.device("default.qubit", wires=n_qubits)
@qml.qnode(dev, interface="torch")
def quantum_convolution_mera(inputs, weights):
    for i in range(n_qubits):
        qml.RY(inputs[i], wires=i) # encodes data

    for i in range(0, n_qubits - 1, 2):
        qml.CNOT(wires=[i, i+1])
        qml.RY(weights[i], wires=i)
        qml.RY(weights[i+1], wires=i+1)

    for i in range(1, n_qubits - 2, 2):
        qml.CNOT(wires=[i, i+1])
        qml.RY(weights[i+2], wires=i)

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

@qml.qnode(dev, interface="torch")
def quantum_error_correction(inputs, weights):
    qml.RY(inputs[0], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])

    # Apply correction: Detect & fix errors
    qml.CNOT(wires=[1, 3])
    qml.CNOT(wires=[2, 4])

    return [qml.expval(qml.PauliZ(i)) for i in range(5)]

# replcing pooling layers based on the paper
@qml.qnode(dev, interface="torch")
def quantum_pooling(inputs):
    for i in range(n_qubits):
        qml.RY(inputs[i], wires=i)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[4, 5])

    return [qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(4))]  # 3 values

# QNN
@qml.qnode(dev, interface="torch")
def quantum_dense(inputs, weights):
    for i in range(len(inputs)):
        qml.RY(inputs[i], wires=i)

    for i in range(len(weights)):
        qml.RY(weights[i], wires=i)

    return qml.expval(qml.PauliZ(0))

class QCNN_MERA_QEC(nn.Module):
    def __init__(self):
        super(QCNN_MERA_QEC, self).__init__()
        self.qconv_weights = nn.Parameter(torch.rand(n_qubits))
        self.qdense_weights = nn.Parameter(torch.rand(3))  # 3 features after pooling

    def forward(self, x):
        x = quantum_convolution_mera(x, self.qconv_weights)
        x = quantum_error_correction(x, self.qconv_weights)
        x = quantum_dense(x, self.qdense_weights)
        return x

In [20]:
inputs = np.load('inputs.npy')
outputs = np.load('output.npy')

print(inputs.shape)
print(outputs.shape)

(1256, 7, 7, 6)
(1256, 1, 1)


In [21]:
batch_size = 16
inputs_tensor = torch.tensor(inputs, dtype=torch.float64)
output_tensor = torch.tensor(outputs, dtype=torch.float64).view(-1, 1)

train_dataset = torch.utils.data.TensorDataset(inputs_tensor, output_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

print(train_dataset)
print(train_loader)

In [23]:
model = QCNN_MERA_QEC()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

for epoch in range(100):
    epoch_loss = 0

    for batch_inputs, batch_labels in train_loader:
        batch_inputs = batch_inputs.view(batch_size, -1)
        optimizer.zero_grad()

        output = model(batch_inputs)
        loss = criterion(output, batch_labels)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/10], Loss: {epoch_loss / len(train_loader):.4f}")

print("Completed ✅")

Epoch [1/10], Loss: 47.3580
Epoch [2/10], Loss: 44.8025
Epoch [3/10], Loss: 44.5629
Epoch [4/10], Loss: 44.5140
Epoch [5/10], Loss: 44.5637
Epoch [6/10], Loss: 44.6063
Epoch [7/10], Loss: 44.5611
Epoch [8/10], Loss: 44.6086
Epoch [9/10], Loss: 44.5753
Epoch [10/10], Loss: 44.5454
Epoch [11/10], Loss: 44.5527
Epoch [12/10], Loss: 44.6094
Epoch [13/10], Loss: 44.5794
Epoch [14/10], Loss: 44.5347
Epoch [15/10], Loss: 44.5939
Epoch [16/10], Loss: 44.4758
Epoch [17/10], Loss: 44.5254
Epoch [18/10], Loss: 44.5823
Epoch [19/10], Loss: 44.6113
Epoch [20/10], Loss: 44.5829
Epoch [21/10], Loss: 44.5405
Epoch [22/10], Loss: 44.6150
Epoch [23/10], Loss: 44.5981
Epoch [24/10], Loss: 44.5725
Epoch [25/10], Loss: 44.5858
Epoch [26/10], Loss: 44.5853
Epoch [27/10], Loss: 44.5812
Epoch [28/10], Loss: 44.5463
Epoch [29/10], Loss: 44.5523
Epoch [30/10], Loss: 44.5681
Epoch [31/10], Loss: 44.5727
Epoch [32/10], Loss: 44.5908
Epoch [33/10], Loss: 44.5729
Epoch [34/10], Loss: 44.5677
Epoch [35/10], Loss: 44